In [1]:
%pip install --upgrade --quiet  langchain langchain-community langchain-openai langchain-ollama langchain-experimental neo4j tiktoken yfiles_jupyter_graphs python-dotenv
%pip install neo4j
%pip install json-repair
%pip install py2neo
%pip install tree_sitter
%pip install --upgrade langchain
%pip install llama-index --upgrade
%pip show llama-index
%pip install yfiles-jupyter-graphs
%pip install --upgrade yfiles-jupyter-graphs
%pip install --upgrade jupyterlab ipywidgets
%pip install jupyterlab
%pip install sqlfluff
%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Name: llama-index
Version: 0.11.15
Summary: Interface between LLMs and your data
Home-page: https://llamaindex.ai
Author: Jerry Liu
Author-email: jerry@llamaindex.ai
License: MIT
Location: /Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages
Requires: llama-index-agent-openai, llama-index-cli, llama-index-core, llama-index-embeddings-openai, llama-index-indices-managed-llama-cloud, llama-index-legacy, llama-index-llms-openai, llama-index-multi-modal-llms-openai, llama-index-program-openai, llama-index-ques

In [2]:
from langchain_core.runnables import  RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field
from langchain_core.output_parsers import StrOutputParser
import os
from langchain_community.graphs import Neo4jGraph
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_community.chat_models import ChatOllama
from langchain_experimental.graph_transformers import LLMGraphTransformer
from neo4j import GraphDatabase
from yfiles_jupyter_graphs import GraphWidget
from langchain_community.vectorstores import Neo4jVector
from langchain_openai import OpenAIEmbeddings
from langchain_community.document_loaders import TextLoader
from langchain_community.vectorstores.neo4j_vector import remove_lucene_chars
import re

from dotenv import load_dotenv

load_dotenv()

/Users/larissakemicloud.com/Library/Python/3.12/lib/python/site-packages/IPython/core/interactiveshell.py:3577: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


True

In [3]:
uri = os.environ["NEO4J_URI"] 
username = os.environ["NEO4J_USERNAME"]
password = os.environ["NEO4J_PASSWORD"]
api_url = os.getenv('API_URL')

# Agora você pode criar o gráfico sem passar os parâmetros diretamente
graph = Neo4jGraph()

In [18]:
import requests
from langchain_openai import ChatOpenAI
from neo4j import GraphDatabase
from langchain_core.prompts import PromptTemplate
from langchain.chains import LLMChain

# Função para executar a consulta Cypher no Neo4j
def execute_cypher_query(query, uri, username, password):
    driver = GraphDatabase.driver(uri, auth=(username, password))
    with driver.session() as session:
        result = session.run(query)
        return [record.data() for record in result]  # Extrair os dados

# Função para executar a query SQL via API
def execute_sql_query(api_url, sql_query):
    headers = {'Content-Type': 'application/json'}
    payload = {'query': sql_query}  # A API provavelmente espera um campo 'query'
    
    response = requests.post(api_url, json=payload, headers=headers)
    
    if response.status_code == 200:
        return response.json()  # Retorna o resultado como JSON
    else:
        return {"error": f"Erro ao executar a query SQL: {response.status_code}, {response.text}"}

# Definir o template para geração de Cypher
CYPHER_GENERATION_TEMPLATE = """Objetivo: Gerar uma instrução Cypher para consultar o banco de dados Neo4J que possui uma estrutura de grafos representando um Schema de Banco de dados SQL.
Instruções:
- Verifique o esquema do banco de dados SQL consultando o Neo4j para obter os nomes das tabelas (nós) e colunas (propriedades).
- Construa uma query Cypher para obter todas as Tabelas e Colunas que relacionem com o Prompt do usuário.
- **Não inclua explicações ou desculpas em suas respostas.**
----------
Exemplo de Query Cypher:
MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'usuario' OR c.id CONTAINS 'usuario' OR c.id CONTAINS 'ativo' OR c.id CONTAINS 'sistema'
RETURN t.id AS tabela, c.id AS coluna
----------
Relação do nome das tabelas existentes dentro do banco de dados Neo4j:
{schema}
----------
User prompt:
{query}
"""

# Conectar ao modelo da OpenAI
llm = ChatOpenAI(temperature=0)

# Criar o template para o prompt de Cypher
schema_data = graph.schema  
user_query = "Quantos usuário ativo existem no sistema?"

# Substituir as variáveis no template com o schema e query do usuário
cypher_prompt = CYPHER_GENERATION_TEMPLATE.format(schema=schema_data, query=user_query)

# Executar o LLM diretamente para gerar a consulta Cypher
cypher_query = llm.predict(cypher_prompt)

# Exibir a consulta Cypher gerada
print(f"Generated Cypher Query: {cypher_query}")

# Executar a consulta Cypher no Neo4j e obter os resultados
cypher_output = execute_cypher_query(cypher_query, uri, username, password)

# Exibir o resultado da execução da query Cypher
print(f"Cypher Query Output: {cypher_output}")

# Definir o template para geração de SQL com base no resultado do Cypher
SQL_TEMPLATE = """Objetivo: Responda sempre com uma query SQL que reflita a pergunta solicitada, sem usar o caractere '*' na construção das queries.

Contexto:
- `t.id` representa o nome da tabela. Por exemplo, se `t.id` for 'Controledeacessousuario', isso significa que o nome da tabela é 'Controledeacessousuario'.
- `c.id` representa o nome da coluna. Por exemplo, se `c.id` for 'Status', isso significa que a coluna que você deve usar na query SQL é 'Status', e você deve verificar por `Status = 1` se a coluna for do tipo `bit`.

Instruções:
- Use somente o nome da tabela e o nome da coluna fornecidos no `Full Context` abaixo para construir a query SQL.
- Se houver uma coluna `Status` do tipo `bit`, inclua uma condição `WHERE` que filtre usuários com `Status = 1` (para ativos).
- Não permita retornos ou queries de 'SELECT *' de uma tabela.
- Use `COUNT()` para contar o número de usuários ativos e dê um alias ao resultado, como `total_usuarios_ativos`.

Full Context: {result}
----------
User prompt:
{query}
"""

# Criar o template para o prompt de SQL
SQL_GENERATION_PROMPT = PromptTemplate(
    input_variables=["result", "query"], template=SQL_TEMPLATE
)

# Agora, use o Full Context (output do Cypher) para gerar uma query SQL correspondente
sql_chain = LLMChain(
    llm=llm,
    prompt=SQL_GENERATION_PROMPT,
    verbose=True
)

# Executar a cadeia para gerar a query SQL com base no output do Cypher
sql_result = sql_chain.run({
    "result": cypher_output, 
    "query": user_query
})

# Exibir a consulta SQL gerada
print(f"SQL gerada: {sql_result}")

# Executar a query SQL na API do SQL Server
sql_output = execute_sql_query(api_url, sql_result)

# Exibir o resultado da execução da query SQL
print(f"SQL Query Output: {sql_output}")

Generated Cypher Query: MATCH (t:Table)-[:HAS_COLUMN]->(c:Column)
WHERE t.id CONTAINS 'usuario' OR c.id CONTAINS 'usuario' OR c.id CONTAINS 'ativo' OR c.id CONTAINS 'sistema'
RETURN t.id AS tabela, c.id AS coluna
Cypher Query Output: [{'tabela': 'Controledeacessousuario', 'coluna': 'Redefinirsenha'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Datadaultimaatualizacao'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Databloqueiousuario'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Email'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Datadecriacao'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Status'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Tipo'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Nomecompleto'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Identificacao'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Senha'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Login'}, {'tabela': 'Controledeacessousuario', 'coluna': 'Id_Empresa'}, 